In [26]:
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, GlobalAveragePooling1D, concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import CSVLogger, ReduceLROnPlateau, ModelCheckpoint 

In [2]:
os.getcwd()

'D:\\next-sentence-predictor'

In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# load document
filename = './finalData/SgCorpus.txt'
doc = load_doc(filename)
print(doc[:505])

who is that smoking hot eurasian/caucasian j1?? i heard he's in waterpolo
i  have not made a confession before.sassy admin: here ya go!
there's this indian girl in neytiri. i think her name starts with a v and she was wearing a blue collegiate yesterday. she's really sweet and cute, especially when she laughs  does anyone know who she is? i really want to get to know her before vday  
why  do not they let the crashers play orientation games this year? suddenly so strict 
anyone knows that sassy india


In [5]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['who', 'is', 'that', 'smoking', 'hot', 'eurasiancaucasian', 'i', 'heard', 'hes', 'in', 'waterpolo', 'i', 'have', 'not', 'made', 'a', 'confession', 'beforesassy', 'admin', 'here', 'ya', 'go', 'theres', 'this', 'indian', 'girl', 'in', 'neytiri', 'i', 'think', 'her', 'name', 'starts', 'with', 'a', 'v', 'and', 'she', 'was', 'wearing', 'a', 'blue', 'collegiate', 'yesterday', 'shes', 'really', 'sweet', 'and', 'cute', 'especially', 'when', 'she', 'laughs', 'does', 'anyone', 'know', 'who', 'she', 'is', 'i', 'really', 'want', 'to', 'get', 'to', 'know', 'her', 'before', 'vday', 'why', 'do', 'not', 'they', 'let', 'the', 'crashers', 'play', 'orientation', 'games', 'this', 'year', 'suddenly', 'so', 'strict', 'anyone', 'knows', 'that', 'sassy', 'indian', 'girl', 'from', 'neytiri', 'she', 'is', 'so', 'cute', 'does', 'anyone', 'know', 'the', 'guy', 'from', 'acs', 'in', 'atom', 'hes', 'plus', 'and', 'is', 'quite', 'fairskinned', 'and', 'skinnyanonymous', 'admin', 'let', 'the', 'orientation', 'confessi

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 3963192


In [8]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
# save sequences to file
out_filename = './finalData/sequences.txt'
save_doc(sequences, out_filename)

In [10]:
filename = './finalData/sequences.txt'
doc = load_doc(filename)
lines = doc.split('\n')

In [14]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [16]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
vocab_size

119694

In [24]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
#y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [30]:
y = to_categorical(y, num_classes=vocab_size)

MemoryError: Unable to allocate 1.73 TiB for an array with shape (3963192, 119694) and data type float32

In [27]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            5984700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 119694)            12089094  
Total params: 18,224,694
Trainable params: 18,224,694
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=1) # epochs=100

ValueError: Error when checking target: expected dense_2 to have shape (119694,) but got array with shape (1,)